## Preprocessing PDF
preprocessing a pdf to textfile. Then combining n lines to create a list of sequences

In [1]:
import PyPDF2
import re
import os

user_file = 'Main_Reference.pdf'
if user_file.endswith('.pdf'):
    pdfFileObj = open(user_file, 'rb')
    pdfReader = PyPDF2.PdfReader(pdfFileObj)
    x = len(pdfReader.pages)

    if os.path.isfile('processedTextFile.txt'):
        os.remove('processedTextFile.txt')
        
    for y in range(x):
        pageObj = pdfReader.pages[y]
        text = pageObj.extract_text()
        with open('processedTextFile.txt', 'a') as f:
            f.write(text)
    user_file = 'processedTextFile.txt'
        

pdfdoc = open(user_file, 'r').read().replace('\n', '')
pdfdoc = re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', pdfdoc)
sequences = list() # list to store multiple lines as sequence
n_sentence = 4
for i in range(0,len(pdfdoc), n_sentence):
    sequences.append(' '.join(pdfdoc[i:i+n_sentence]))
# sequences = list(sequences)

In [2]:
print(sequences[20])

This might be as a result of the orientation of the captured fingerprint images. The fingerprints were captured using various angles of the scanner, as it depends on where the subject stands before the scanner for capturing. Moreover, 833 right hands were correctly classified, with 92.56% overall classification rate,where 67 right hands were wrongly misclassified as left hands. In Table III, individual subject s’fingers are classified with an overall accuracy of 76.72%.


In [3]:
from transformers import RobertaTokenizer, RobertaModel

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaModel.from_pretrained("roberta-base")

vectors = [
    # tokenize the sequence, return it as PyTorch tensors (vectors),
    # and pass it onto the model
    model(**tokenizer(sequence, return_tensors='pt'))[0].detach().squeeze()
    for sequence in sequences
]
print(vectors[0])

/home/csa/miniconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[-0.0110,  0.0994, -0.0149,  ..., -0.1003, -0.0117, -0.0279],
        [ 0.1471,  0.2742,  0.0348,  ..., -0.3416,  0.0040,  0.0922],
        [ 0.0233,  0.3932,  0.0820,  ...,  0.1234,  0.3126,  0.0336],
        ...,
        [ 0.0501,  0.0220,  0.0967,  ..., -0.1937,  0.2379,  0.2083],
        [-0.0063,  0.0939, -0.0430,  ..., -0.1464, -0.0137, -0.0600],
        [-0.0034,  0.1189, -0.0209,  ..., -0.0830,  0.0084,  0.0249]])


In [4]:
import torch

def encode(document: str) -> torch.Tensor:
    tokens = tokenizer(document, return_tensors='pt')
    vector = model(**tokens)[0].detach().squeeze()
    return torch.mean(vector, dim=0)

## Top K similar Search Function

In [5]:
import faiss
import numpy as np
import torch

averaged_vectors = [torch.mean(vector, dim=0) for vector in vectors]

v_size = [v.size() for v in averaged_vectors][0][0]
index = faiss.IndexIDMap(faiss.IndexFlatIP(v_size)) # the size of our vector space
# index all the documents, we need them as numpy arrays first
index.add_with_ids(
    np.array([t.numpy() for t in averaged_vectors]),
    # the IDs will be 0 to len(documents)
    np.array(range(0, len(sequences)))
)

def search(query: str, k=1):
    encoded_query = encode(query).unsqueeze(dim=0).numpy()
    top_k = index.search(encoded_query, k)
    scores = top_k[0][0]
    results = [sequences[_id] for _id in top_k[1][0]]
    return list(zip(results, scores))

In [6]:
with open('userstring.txt', 'r') as f:
    user_text = f.read()
    
results = search(user_text, k=3)
for result in results:
    print(f'Similarity score: {result[1]}\nText:')
    print(result[0])

Similarity score: 165.63258361816406
Text:
And it has been proven by many researchers that fingerprints can be used for gender classification, which is helpful when short-listing suspects [4-7]. The uniqueness of the fingerprint features can reduce the gender identification difficulties and limit the amount of time it takes to identify a suspect. These unique features of fingerprints can be used in differentiating between individualsby their gender and, therefore, it makes faster theidentification processes of unknown suspects. Furthermore, itcan guide forensic investigators to the correct identity of a suspect when matching the suspect’s fingerprints among the large number of possible matches in the fingerprint databases.It is extremely time consuming to identify and detect unknown fingerprints in a large volume of fingerprint databases during investigation.
Similarity score: 165.5492401123047
Text:
Similarly, for hand identification, there are 3000 images from left hands and 3000 fro